# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, WT, Female, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISp and VISl sessions. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('wt/wt') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."


In [7]:
session_id = filtered_sessions.index.values[0]

session = cache.get_session_data(session_id)

Downloading:   0%|          | 0.00/2.91G [00:00<?, ?B/s]

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

/Users/tulsipatel/.local/lib/python3.8/site-packages/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_warehouse_api.py:291: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  pv_is_bool = np.issubdtype(output["p_value_rf"].values[0], np.bool)


Total number of units: 824


,waveform_velocity_below,amplitude_cutoff,waveform_repolarization_slope,snr,firing_rate,waveform_duration,presence_ratio,isi_violations,cumulative_drift,waveform_spread,max_drift,waveform_PT_ratio,nn_hit_rate,waveform_recovery_slope,waveform_velocity_above,isolation_distance,L_ratio,peak_channel_id,d_prime,cluster_id,waveform_amplitude,waveform_halfwidth,local_index_unit,silhouette_score,nn_miss_rate,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
915956356,NaN,0.054706,0.632977,2.605408,2.193113,0.549414,0.99,0.430427,263.01,60.0,18.37,0.512847,0.637363,-0.106051,-0.451304,18.136302,0.061345,850229419,2.936851,14,214.954545,0.233501,14,0.108867,0.000673,NaN,1100.0,1.699291,0.776937,0.920000,0.650000,1.452569,0.592722,0.047321,0.017619,0.039495,53.636,0.204814,0.0,0.04,15.0,0.332813,-0.142857,-0.077160,NaN,0.595238,315.0,60.0,0.106585,0.749752,0.770016,NaN,0.029437,-15.455,4992,0.75,2.247928,1.438807,2.613584,1.564701,2.218146,NaN,0.0575,False,False,False,False,False,-0.046051,False,0.114104,0.241255,0.056767,0.416234,0.067618,40,27,733744647,3,8.0,grey,NaN,NaN,NaN,probeB,See electrode locations,29999.915391,1249.996475,True
915956349,NaN,0.096378,0.553393,2.684636,16.192413,0.508208,0.99,0.120715,169.29,40.0,14.87,0.424620,0.874667,-0.086022,0.206030,54.635515,0.010406,850229419,2.973959,13,177.115380,0.192295,13,0.136901,0.021636,NaN,600.0,5.764529,0.877288,1.189189,1.268914,2.335358,0.288541,0.015031,0.040988,0.087448,58.333,0.960864,0.0,0.02,15.0,-0.032581,0.143963,0.006477,NaN,0.137202,180.0,90.0,0.854594,0.416630,0.955454,NaN,0.294999,-20.000,4944,0.00,16.046763,7.247322,17.963269,9.209478,15.128685,NaN,0.0705,False,False,False,False,False,0.323068,False,0.095038,0.166625,0.028184,0.073108,0.029319,40,27,733744647,3,8.0,grey,NaN,NaN,NaN,probeB,See electrode locations,29999.915391,1249.996475,True
915956345,NaN,0.001703,0.781296,3.362198,12.698430,0.521943,0.99,0.044936,129.36,30.0,11.56,0.500520,0.930000,-0.153127,0.343384,47.805714,0.008281,850229419,3.559911,12,251.275830,0.192295,12,0.138827,0.004956,NaN,600.0,4.236128,1.643568,1.817460,0.990123,3.421208,0.268001,0.038387,0.040152,0.085168,40.000,0.123119,0.0,0.04,8.0,0.222727,0.006618,-0.096483,-0.081081,0.431100,225.0,60.0,0.178474,0.982081,0.509995,0.545245,0.020251,-25.000,5010,0.00,11.973721,7.860148,15.618365,12.267608,10.896224,NaN,0.0725,False,False,False,False,False,0.179102,False,0.086342,0.337854,0.045184,0.302351,0.062177,40,27,733744647,3,8.0,grey,NaN,NaN,NaN,probeB,See electrode locations,29999.915391,1249.996475,True
915956340,NaN,0.000881,0.704522,3.357908,9.660554,0.563149,0.99,0.006825,160.24,30.0,7.79,0.439372,0.987654,-0.104196,0.000000,59.613182,0.000410,850229419,5.519024,11,247.254345,0.206030,11,0.197458,0.000903,NaN,600.0,13.657938,1.428046,3.537971,4.437255,4.910479,0.419164,0.094477,0.145318,0.075828,25.000,0.209387,0.0,0.02,8.0,0.355659,0.207353,0.397513,-0.054545,0.651261,270.0,90.0,0.209249,0.435825,0.028048,0.931928,0.00075

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 74


,waveform_velocity_below,amplitude_cutoff,waveform_repolarization_slope,snr,firing_rate,waveform_duration,presence_ratio,isi_violations,cumulative_drift,waveform_spread,max_drift,waveform_PT_ratio,nn_hit_rate,waveform_recovery_slope,waveform_velocity_above,isolation_distance,L_ratio,peak_channel_id,d_prime,cluster_id,waveform_amplitude,waveform_halfwidth,local_index_unit,silhouette_score,nn_miss_rate,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
915960297,0.686767,0.001235,0.335321,2.711213,3.051031,0.659296,0.99,0.089814,214.70,60.0,43.69,0.410785,0.853659,-0.069308,0.618090,50.575739,0.006127,850231281,3.013238,335,117.866385,0.178559,305,0.028954,0.002203,NaN,2400.0,0.749020,2.013333,1.719770,8.679167,0.660635,0.339622,0.041067,0.360863,0.097006,42.917,1.086793,0.080,0.32,1.0,0.116822,0.742188,0.235294,-0.566667,0.446581,0.0,0.0,0.366925,0.078382,0.352808,0.460164,0.013160,1.250,4916,0.75,2.753605,1.651963,3.102582,3.717125,2.683756,NaN,0.2425,False,False,False,False,False,0.417966,False,0.211299,0.546142,0.074951,0.067780,0.131217,2460,11,733744649,245,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.991665,1249.999653,True
915960294,0.686767,0.000761,0.353458,2.454865,13.198225,0.549414,0.99,0.006399,159.52,60.0,48.98,0.210040,0.986000,-0.022002,0.412060,74.417854,0.002733,850231281,3.250551,334,106.159365,0.219765,304,0.085764,0.003794,NaN,2600.0,2.454559,0.535976,0.995266,0.352381,1.010163,0.271556,0.016422,0.151408,0.008670,46.538,0.890061,0.000,0.02,8.0,NaN,-0.114187,-0.186014,NaN,-0.659908,180.0,0.0,NaN,0.370739,0.145697,NaN,0.000340,-20.000,5014,0.50,11.394742,17.079168,11.238882,16.160172,10.569164,NaN,0.0405,False,False,False,False,False,0.074237,False,0.086745,0.003038,0.037601,0.028354,0.017555,2460,11,733744649,245,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.991665,1249.999653,True
915960290,0.000000,0.053858,0.479279,2.727632,5.642346,0.219765,0.99,0.038768,491.48,20.0,52.31,0.248721,0.832237,-0.123698,NaN,58.246321,0.008143,850231281,2.292449,333,131.387490,0.068677,303,0.004020,0.006807,NaN,300.0,8.478431,0.880000,5.377279,1.628070,6.937295,0.901675,0.119627,0.161230,0.156122,86.667,0.137576,0.007,0.32,1.0,-0.378788,0.176471,0.280503,0.259259,0.275862,180.0,0.0,0.402665,0.869759,0.142947,0.637922,0.399941,16.667,4916,0.00,3.150392,0.346379,12.212847,1.475885,6.170841,NaN,0.0805,False,False,False,False,False,0.403915,False,0.137210,0.257732,0.292129,0.223157,0.180916,2460,11,733744649,245,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.991665,1249.999653,True
915960287,0.000000,0.034125,0.462609,2.925401,14.034699,0.151089,0.99,0.005457,309.50,40.0,57.26,0.560337,0.926000,-0.279772,-0.686767,67.692919,0.012434,850231281,3.832008,332,153.604815,0.082412,302,0.008405,0.005862,NaN,800.0,17.614296,1.020000,5.089333,2.910032,4.976616,0.502050,0.055007,0.153380,0.038126,18.750,2.079195,0.793,0.32,1.0,0.094737,-0.116883,-0.297941,0.502825,0.552379,180.0,30.0,0.8

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
915960297     2.683756
915960294    10.569164
915960290     6.170841
915960287    15.601623
915960283     4.529547
               ...    
915961119     0.478933
915960933     0.119234
915961131     0.277102
915960968     0.369025
915960996     0.530889
Name: firing_rate_sg, Length: 74, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

8.055310590583035


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
915960297     2.753605
915960294    11.394742
915960290     3.150392
915960287    10.794515
915960283     3.912096
               ...    
915961119     0.521082
915960933     0.075427
915961131     0.120045
915960968     0.161477
915960996     0.074896
Name: firing_rate_dg, Length: 74, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

6.7662075825115044


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])